In [1]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'
group_dir_ = '../data/groups/low'
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'

In [2]:
predictions = pd.read_pickle(os.path.join(dir_, 'post_filtering (count)', str(0.005) + '_prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))

In [3]:
predictions[:5]

,uid,tid,rating
6,0,48,2.821581
10,0,59,2.449405
11,0,70,3.864443
14,0,83,3.358146
32,0,176,2.833267


In [4]:
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,rating,count
26,0,9,4.75,15
37,0,15,2.50,3
45,0,22,3.50,6
64,0,28,2.75,4
75,0,33,0.25,1


In [5]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)
    print(len(groups[0]))

2
3
4
5
6
7
8


In [6]:
num_user = len(predictions['uid'].unique())
num_user

953

In [7]:
precision_4 = []
precision_20 = []
precision_50 = []

recall_4 = []
recall_20 = []
recall_50 = []

In [8]:
# Precision and Recall
top_n_size = 50
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([50, 20, 4]):
    avg_precisions = []
    avg_recalls = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        recalls =[]
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            
            
            for k in range(len(group)):
                high_rating = 0
                for l in range(top_n_size):
                    uid = group[k]
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1
                        
                precision = high_rating / top_n_size
                recall = high_rating / len(test[test['uid'] == uid])
                precisions.append(precision)
                recalls.append(recall)
                
        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)
        
        avg_recall = 0
        for recall in recalls:
            avg_recall += recall
        avg_recall /= len(recalls)
        avg_recalls.append(avg_recall)
        
    if top_n_size == 4:
        precision_4.append(avg_precisions)
    elif top_n_size == 20:
        precision_20.append(avg_precisions)
    elif top_n_size == 50:
        precision_50.append(avg_precisions)
        
    if top_n_size == 4:
        recall_4.append(avg_recalls)
    elif top_n_size == 20:
        recall_20.append(avg_recalls)
    elif top_n_size == 50:
        recall_50.append(avg_recalls)


In [9]:
for i in precision_4:
    print(i)

for i in precision_20:
    print(i)

for i in precision_50:
    print(i)

for i in recall_4:
    print(i)

for i in recall_20:
    print(i)

for i in recall_50:
    print(i)

[0.21087257617728533, 0.15618448637316562, 0.11082474226804123, 0.10757575757575757, 0.07552083333333333, 0.07417582417582418, 0.07762096774193548]
[0.15166204986149528, 0.11226415094339601, 0.08311855670103113, 0.07272727272727293, 0.061458333333333455, 0.050183150183150234, 0.04717741935483878]
[0.1245706371191137, 0.08922431865828095, 0.06927835051546355, 0.05806060606060581, 0.048263888888888745, 0.04036630036630021, 0.034919354838709574]
[0.001561219124474918, 0.0013659530961314915, 0.0014190112590963372, 0.0013817776055043947, 0.0010103785666236741, 0.0016056510812328243, 0.0014571971255677454]
[0.005458775913642639, 0.005291746834809949, 0.0043736711112748825, 0.00465160192461916, 0.004256117052841385, 0.0045479062339554635, 0.004204080521181675]
[0.0109637136862546, 0.009956209529739215, 0.009592836011807122, 0.009189882824749253, 0.008550410479777034, 0.008753107247732561, 0.00801715774260562]


In [10]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    for i in range(len(r)):
        r[i] = 2**r[i] -1
    
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, r_max, k, method=0):
    dcg_max = dcg_at_k(r_max, k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [11]:
nDCG_4 = []
nDCG_20 = []
nDCG_50 = []

In [12]:
# Precision and Recall
top_n_size = 50
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([50, 20, 4]):
    avg_nDCGs = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        nDCGs = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            
            
            for k in range(len(group)):
                truth_rating = []
                uid = group[k]
                for l in range(top_n_size):
                    tid = top_n_list[l]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    
                    if len(t) > 0:
                        truth_rating.append(t.iloc[0]['rating'])
                    else:
                        truth_rating.append(0)
                max_rating = test[test['uid']==uid].sort_values(by=['rating'],  ascending=False)['rating'].values[:top_n_size]
                nDCG = ndcg_at_k(truth_rating, max_rating, top_n_size, method=1)
                nDCGs.append(nDCG)                
        avg_nDCG = 0
        for nDCG in nDCGs:
            avg_nDCG += nDCG
        avg_nDCG /= len(nDCGs)
        avg_nDCGs.append(avg_nDCG)
        
    if top_n_size == 4:
        nDCG_4.append(avg_nDCGs)
    elif top_n_size == 20:
        nDCG_20.append(avg_nDCGs)
    elif top_n_size == 50:
        nDCG_50.append(avg_nDCGs)

In [13]:
for i in nDCG_4:
    print(i)

for i in nDCG_20:
    print(i)

for i in nDCG_50:
    print(i)

[0.12891156959142935, 0.08938114651504163, 0.062109764239084556, 0.05251061771688512, 0.03317087661739261, 0.03455340704791771, 0.03586328289191286]
[0.09264298413621495, 0.0651998898833028, 0.04840289596933937, 0.03967122757362344, 0.030390245936744862, 0.027226798823425394, 0.02576132300444219]
[0.07591816382819747, 0.0543335626119103, 0.04184892003929762, 0.035255525104092605, 0.026389404799875213, 0.024622294235847404, 0.021847746278219674]
